## Calculate the NPC diameter is stress conditions from a line profile acquired from Image J 

The idea is to see if there is any difference in the NPC diameter depending on the stress applied to the cell before fixation. Before this analysis, ON events are detected with Picasso Localize and localizations are reconstructed with Picasso Render. A drift correction is then performed on Picasso Render. The rendered localizations are exported on a .csv file and gave to ThunderStorm plugin on Image J. ThunderStorm reconstructs the image.  

Workflow of the NPC diameter analysis: 
- Single NPC is picked 
- A line profile is traced on the chosen NPC 
- Line profile data set is saved on a .csv file 

### Import the packages

# Line profile good method

The idea of this code is to compare NPC diameter when cells are under mechanical constraints or stress. 

We start from a DNA PAINT reconstructed image and from x,y coordinates of NPC centers (manually picked in Jupyter) and we want to extract NPC diameter for each NPC in each condition. 

### Import packages

In [ ]:
from tifffile import tifffile
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np 
from line_profile import generate_line_profile, generate_rotational_line_profile, average_rotational_line_profile

### Load DNA PAINT reconstructed images 

The localizations were detected in Picasso Localize. Then the drift correction and needed filtering was done in Picasso Render and Picasso Filter respectively. Localizations were exported in a txt file and opened through the ThunderStorm plugin of ImageJ. From the localizations, a reconstructed image is produced. 

In [ ]:
# Load control condition image
nucleus_control = "C:/Users/Emilie/Documents/Data/DNA_PAINT/2023/20231204/Fileset_879751/20231204_reconstructed_image_thunderstorm_3.tif"

# Load hypertonic condition image 
nucleus_hyper = "C:/Users/Emilie/Documents/Data/DNA_PAINT/2023/20231213/Fileset_882048/20231213_reconstructed_image_thunderstorm.tif"

# Load hypotonic condition image 
#nucleus_hypo = "C:/Users/Emilie/Documents/Data/DNA_PAINT/2023/20231213/Fileset_882048/20231213_reconstructed_image_thunderstorm.tif"

# Opens a image in RGB mode
image_control = Image.open(nucleus_control)
image_hyper = Image.open(nucleus_hyper)
#image_hypo = Image.open(nucleus_hypo)

with tifffile.TiffFile(nucleus_control) as tif:
    raw_control = tif.pages[0].asarray()  # image as numpy array
    pxs_nm_control = 1e9/tif.pages[0].tags['XResolution'].value[0]  # pixel size in nm
    print('px size= ',pxs_nm_control)
    print(raw_control.shape)
    
with tifffile.TiffFile(nucleus_hyper) as tif:
    raw_hyper = tif.pages[0].asarray()  # image as numpy array
    pxs_nm_hyper = 1e9/tif.pages[0].tags['XResolution'].value[0]  # pixel size in nm
    print('px size= ',pxs_nm_hyper)
    print(raw_hyper.shape)

'''with tifffile.TiffFile(nucleus_hypo) as tif:
    raw_hypo = tif.pages[0].asarray()  # image as numpy array
    pxs_nm_hypo = 1e9/tif.pages[0].tags['XResolution'].value[0]  # pixel size in nm
    print('px size= ',pxs_nm_hypo)
    print(raw_hypo.shape)'''
    

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Reconstructed DNA PAINT image control')
ax.imshow(image_control)

### Automatic NPC detection 

1. Gaussian blur applied on the image --> the goal is to transform a ring into a bright spot
2. Peak detection + centroïd finding 

In [ ]:
from scipy.ndimage import gaussian_filter
from nucmask import nucleus_mask, nucleus_focus_mask
    
# Gaussian Blur 
im_ctrl_blur = gaussian_filter(raw_control, (5,5))
fig, ax = plt.subplots()
ax.set_title('Blured DNA PAINT image control')
ax.imshow(im_ctrl_blur)

# Nucleus mask to remove out of focus regions 
mask_ctrl = nucleus_focus_mask(im_ctrl_blur, gaussstd_nm=500, pxs_nm=9, num_di=12, num_er=5) # get binary mask 
processed_control_img = (im_ctrl_blur)*mask_ctrl
fig, ax = plt.subplots()
ax.set_title('Blured DNA PAINT image control + mask')
ax.imshow(processed_control_img)

In [ ]:
# Automatic NPC detection 

from detect import detect_pores
from skimage.filters import threshold_otsu

Th_control = threshold_otsu(raw_control)+np.std(raw_control)
print(Th_control)
control_peaks = detect_pores(processed_control_img, threshold=Th_control, fwhm=120/pxs_nm_control)

fig, ax = plt.subplots(figsize=(12,15))
ax.scatter(control_peaks[:,0], control_peaks[:,1], s=20,c='#FF000000',marker='o',edgecolors='y',linewidths=1)
ax.set_title('Detected NPC on DNA PAINT image control')
ax.imshow(image_control)

In [ ]:
# For each detected NPC store the x and y coordinates of the center 

# create an Empty DataFrame object
df = pd.DataFrame()
 
# append columns to an empty DataFrame
df['x_coordinate'] = control_peaks[:,0]
df['y_coordinate'] = control_peaks[:,1]
df['Condition']= "Control"
 
df


In [ ]:
#crop_img=[]
#diameter = []
#diameter_hypertonic = []
# Function to crop around the NPC center
def center_crop(img,dim,cent_x,cent_y):  
    width, height = img.shape[1], img.shape[0]

    # process crop width and height for max available dimension
    crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
    crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
    cw2, ch2 = int(crop_width/2), int(crop_height/2) 
    crop_img = img[int(cent_y)-ch2:int(cent_y)+ch2, int(cent_x)-cw2:int(cent_x)+cw2]
    return (crop_img)

In [ ]:
# Define the parameters for rotational line profile

angle_bin = 20

length_control = round(300/pxs_nm_control)
length_hyper = round(300/pxs_nm_hyper)
#length_hypo = round(300/pxs_nm_hypo)

bins = 51

l_control=round(length_control/2)+1
l_hyper = round(length_hyper/2)+1
# l_hypo = round(length_hypo/2)+1

x = np.linspace(-150,150,bins) 

In [ ]:
# Do it for all the NPC 

crop_control = []#; crop_hyper = []#; crop_hypo = []
profile_control = []#; profile_hyper = []#; profile_hypo = [] 
average_control = []#; average_hyper = []#; average_hypo = []

for i in range(0,len(df['x_coordinate'])):
    if (df['Condition'].iloc[i]=='Control'):
        crop_control.append(center_crop(raw_control,(20,20),df['x_coordinate'][i],df['y_coordinate'][i]))
        profile_control.append(generate_rotational_line_profile(raw_control, (df['x_coordinate'][i],df['y_coordinate'][i]), 
                                                       angle_bin, length_control, bins))
        average_control.append(average_rotational_line_profile(raw_control, (df['x_coordinate'][i],df['y_coordinate'][i]), 
                                                       angle_bin, length_control, bins))
    
    '''if (df['Condition'].iloc[i]=='Hypertonic'):
        crop_hyper.append(center_crop(raw_hyper,(20,20),df['x_px'][i],df['y_px'][i]))
        profile_hyper.append(generate_rotational_line_profile(hyper_img, (df['x_px'][i], df['y_px'][i]), 
                                                       angle_bin, length_control, bins))
        average_hyper.append(average_rotational_line_profile(hyper_img, (df['x_px'][i],df['y_px'][i]), 
                                                       angle_bin, length_control, bins))'''
    '''if (df['Condition'].iloc[i]=='Hypotonic'):
        crop_hypo.append(center_crop(raw_hypo,(20,20),df['x_px'][i],df['y_px'][i]))
        profile_hypo.append(generate_rotational_line_profile(hypo_img, (df['x_px'][i], df['y_px'][i]), 
                                                       angle_bin, length_control, bins))
           average_hypo.append(average_rotational_line_profile(hypo_img, (df['x_px'][i],df['y_px'][i]), 
                                                       angle_bin, length_control, bins))'''

for k in range (0, len(profile_control)):
# One can do the following part in a loop to print at the same time all the crop and all the profiles 
    fig,(ax0,ax1) = plt.subplots(nrows=1, ncols=2,figsize=(8,4))
# Plot line profiles for each channel
    ax0.imshow(crop_control[k])
    for angle in range(0,int(180-180/angle_bin),int(180/angle_bin)):
        X = np.linspace(10 - (length_control/3) * np.cos(angle),
                        10 + (length_control/3) * np.cos(angle),
                        bins)
        Y = np.linspace(10 - (length_control/3) * np.sin(angle),
                        10 + (length_control/3) * np.sin(angle),
                        bins)
        ax0.plot(X,Y)
    ax1.plot(x,average_control[k],'g',linewidth=3)
    for j in range(0,len(profile_control[k])):
        ax1.plot(x,profile_control[k][j],'black',linestyle='dashed')

#### Filter the line profiles that only have 2 peaks 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel
from scipy.signal import find_peaks

# In this part I try to filter the profiles that only have 2 peaks 

x = np.linspace(-150,150,bins) 

def count_peaks(x, y):
    # Use scipy's find_peaks to identify peaks in the signal
    peaks = find_peaks(y, height=20)  # Adjust the height parameter as needed
    return len(peaks)

filtered_profiles = []
def filter_profiles(profiles):
    # Filter out profiles with a number of peaks different from 2
    for profile in profiles:
        if count_peaks(x,df['y_coordinate'])==2:
            filtered_profiles.append(profile)
    return filtered_profiles

nb_peaks = []
for i in range(0,len(average_control)):
    nb_peaks.append(count_peaks(x,df['y_coordinate']))
print(nb_peaks)

#### Fit with a two gaussian function to calculate the NPC diameter

In [ ]:
#Fit the average profiles with a two gaussian function 

from scipy.optimize import curve_fit
x = np.linspace(-150,150,bins) 
diameter = []
NPC_ID = []
# Define a two gaussian function
def two_gaussians(x, a1, b1, c1, a2, b2, c2):
    return a1 * np.exp(-(x - b1)**2 / (2 * c1**2)) + a2 * np.exp(-(x - b2)**2 / (2 * c2**2))

# Define a function to fit with a two gaussian curve 
def two_gauss_fit(initial_guess,profile):
    # Fit the data using curve_fit
    for k in range(0,len(filter_profiles(profile))):
        try: 
            my_array=filter_profiles(profile[k])
            if all(element == my_array[0] for element in my_array):
                    # If all values are 0, delete the array
                del my_array

            else:
                params, covariance = curve_fit(two_gaussians, x, filter_profiles(profile[k]), p0=initial_guess)

                    # Extract the fitted parameters
                a1_fit, b1_fit, c1_fit, a2_fit, b2_fit, c2_fit = params

                    # Generate the fitted curve
                y_fit = two_gaussians(x, a1_fit, b1_fit, c1_fit, a2_fit, b2_fit, c2_fit)

                diameter.append(-params[1]+params[4])
                NPC_ID.append(k)
        except: 
            pass
    return(diameter,NPC_ID)

diam_ctrl = two_gauss_fit((1e4, -50, 20, 5e4, 50, 20),filter_profiles(average_control))
dfdiam = pd.DataFrame()
dfdiam['NPC diameter'] = diam_ctrl[0]
dfdiam['NPC id'] = diam_ctrl[1]
dfdiam['Condition'] = 'Control'
print(dfdiam)

'''diam_hyper = two_gauss_fit((1e4, -50, 20, 5e4, 50, 20),profile_hyper)
dfdiam_hyper = pd.DataFrame()
dfdiam_hyper['NPC diameter'] = diam_hyper[0]
dfdiam_hyper['NPC id'] = diam_hyper[1]
dfdiam_hyper['Condition'] = 'Hypertonic'
#print(dfdiam_hyper)

df_final = pd.concat([dfdiam,dfdiam_hyper],ignore_index=True)
print(df_final)'''

In [ ]:
print(np.mean(dfdiam['NPC diameter']))

In [ ]:
# Maybe not needed 

from process import process
from nucmask import nucleus_mask, nucleus_focus_mask
# Image processing 
# Gaussian size in nm for denoising
gf_nm = 10 # sigma for gaussian blur 

control_img = process(raw_control, gf_nm, pxs_nm_control)
#hypo_img = process(raw_hypo, gf_nm, pxs_nm_hypo)
hyper_img = process(raw_hyper, gf_nm, pxs_nm_hyper)

mask_ctrl = nucleus_focus_mask(control_img, gaussstd_nm=500, pxs_nm=15, num_di=12, num_er=5) # get binary mask 
control_img = (control_img)*mask_ctrl
#mask_hypo = nucleus_focus_mask(hypo_img, gaussstd_nm=500, pxs_nm=15, num_di=12, num_er=5) # get binary mask 
#hypo_img = (hypo_img)*mask_hypo
mask_hyper = nucleus_focus_mask(hyper_img, gaussstd_nm=500, pxs_nm=15, num_di=12, num_er=5) # get binary mask
hyper_img = (hyper_img)*mask_hyper

fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(12, 15))
axs[0].set_title('Reconstructed DNA PAINT image control')
axs[0].imshow(control_img)

axs[1].set_title('Reconstructed DNA PAINT image hypertonic')
axs[1].imshow(hyper_img)
plt.show()

### Non automatic version do detect pores 

In [ ]:
NPC_centers_control = "C:/Users/Emilie/Documents/Data/DNA_PAINT/2023/20231204/Fileset_879751/NPC_centers.ods"
NPC_centers_hyper = "C:/Users/Emilie/Documents/Data/DNA_PAINT/2023/20231213/Fileset_882048/NPC_centers.ods"
#NPC_centers_hypo = 

# Transform the coordinate into pixel number 
df_control = pd.read_excel(NPC_centers_control, engine='odf')
condition = 'Control'
df_control['Condition'] = condition
x_nm = df_control['x_um']*1000
y_nm = df_control['y_um']*1000
x_px = x_nm/pxs_nm_control # Need to distingwish between control and other conditions 
y_px = y_nm/pxs_nm_control
df_control['x_px']=x_px
df_control['y_px']=y_px

df_hyper = pd.read_excel(NPC_centers_hyper, engine='odf')
condition = 'Hypertonic'
df_hyper['Condition'] = condition
x_nm = df_hyper['x_um']*1000
y_nm = df_hyper['y_um']*1000
x_px = x_nm/pxs_nm_hyper # Need to distingwish between control and other conditions 
y_px = y_nm/pxs_nm_hyper
df_hyper['x_px']=x_px
df_hyper['y_px']=y_px

'''df_hypo = 
condition = 'Hypotonic'
df_hypo['Condition'] = condition'''

# Concatenate the 3 dataframes
#df = pd.concat([df_control,df_hyper,df_hypo],ignore_index = True)
df = pd.concat([df_control,df_hyper],ignore_index = True)

print(df)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(control_img)
x = df['x_px'][df['Condition'] == 'Control']
y = df['y_px'][df['Condition'] == 'Control']
ax.scatter(x, y, s=10,c='m', marker='+',linewidths=2)
ax.set_title('Reconstructed DNA PAINT image control condition', fontsize=10)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plt.imshow(hyper_img)
x = df['x_px'][df['Condition'] == 'Hypertonic']
y = df['y_px'][df['Condition'] == 'Hypertonic']
ax.scatter(x, y, s=10,c='m', marker='+',linewidths=2)
ax.set_title('Reconstructed DNA PAINT image hypertonic condition', fontsize=10)

In [ ]:
#crop_img=[]
#diameter = []
#diameter_hypertonic = []
# Function to crop around the NPC center
def center_crop(img,dim,cent_x,cent_y):  
    width, height = img.shape[1], img.shape[0]

    # process crop width and height for max available dimension
    crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
    crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
    cw2, ch2 = int(crop_width/2), int(crop_height/2) 
    crop_img = img[int(cent_y)-ch2:int(cent_y)+ch2, int(cent_x)-cw2:int(cent_x)+cw2]
    return (crop_img)

In [ ]:
# Define the parameters for rotational line profile

angle_bin = 20

length_control = round(300/pxs_nm_control)
length_hyper = round(300/pxs_nm_hyper)
#length_hypo = round(300/pxs_nm_hypo)

bins = 51

l_control=round(length_control/2)+1
l_hyper = round(length_hyper/2)+1
# l_hypo = round(length_hypo/2)+1

x = np.linspace(-150,150,bins) 

In [ ]:
# Do it for all the NPC 

crop_control = []; crop_hyper = []#; crop_hypo = []
profile_control = []; profile_hyper = []#; profile_hypo = [] 
average_control = []; average_hyper = []#; average_hypo = []

for i in range(0,len(df['x_px'])):
    if (df['Condition'].iloc[i]=='Control'):
        crop_control.append(center_crop(raw_control,(20,20),df['x_px'][i],df['y_px'][i]))
        profile_control.append(generate_rotational_line_profile(control_img, (df['x_px'][i],df['y_px'][i]), 
                                                       angle_bin, length_control, bins))
        average_control.append(average_rotational_line_profile(control_img, (df['x_px'][i],df['y_px'][i]), 
                                                       angle_bin, length_control, bins))
    
    if (df['Condition'].iloc[i]=='Hypertonic'):
        crop_hyper.append(center_crop(raw_hyper,(20,20),df['x_px'][i],df['y_px'][i]))
        profile_hyper.append(generate_rotational_line_profile(hyper_img, (df['x_px'][i], df['y_px'][i]), 
                                                       angle_bin, length_control, bins))
        average_hyper.append(average_rotational_line_profile(hyper_img, (df['x_px'][i],df['y_px'][i]), 
                                                       angle_bin, length_control, bins))
    if (df['Condition'].iloc[i]=='Hypotonic'):
        '''crop_hypo.append(center_crop(raw_hypo,(20,20),df['x_px'][i],df['y_px'][i]))'''
        '''profile_hypo.append(generate_rotational_line_profile(hypo_img, (df['x_px'][i], df['y_px'][i]), 
                                                       angle_bin, length_control, bins))
           average_hypo.append(average_rotational_line_profile(hypo_img, (df['x_px'][i],df['y_px'][i]), 
                                                       angle_bin, length_control, bins))'''

for k in range (0, len(profile_hyper)):
# One can do the following part in a loop to print at the same time all the crop and all the profiles 
    fig,(ax0,ax1) = plt.subplots(nrows=1, ncols=2,figsize=(8,4))
# Plot line profiles for each channel
    ax0.imshow(crop_hyper[k])
    for angle in range(0,int(180-180/angle_bin),int(180/angle_bin)):
        x = np.linspace(10 - (length_hyper/2) * np.cos(angle),
                        10 + (length_hyper/2) * np.cos(angle),
                        bins)
        y = np.linspace(10 - (length_hyper/2) * np.sin(angle),
                        10 + (length_hyper/2) * np.sin(angle),
                        bins)
        ax0.plot(x,y)
        print(angle)
    ax1.plot(x,average_hyper[k],'g',linewidth=3)
    for j in range(0,len(profile_hyper[k])):
        ax1.plot(x,profile_hyper[k][j],'black',linestyle='dashed')
        
print(len(profile_hyper[0]))

In [ ]:
print(len(profile_hyper[15]))

In [ ]:
initial_guess = [1e4, -50, 20, 5e4, 50, 20] # Initial guesses for the parameters

In [ ]:
from scipy.optimize import curve_fit
x = np.linspace(-150,150,bins) 
diameter = []
NPC_ID = []
# Define a two gaussian function
def two_gaussians(x, a1, b1, c1, a2, b2, c2):
    return a1 * np.exp(-(x - b1)**2 / (2 * c1**2)) + a2 * np.exp(-(x - b2)**2 / (2 * c2**2))

# Define a function to fit with a two gaussian curve 
def two_gauss_fit(initial_guess,profile):
    # Fit the data using curve_fit
    for k in range(0,len(profile)):
        for j in range(0,len(profile[k])):
            try: 
                my_array=profile[k][j]
                if all(element == my_array[0] for element in my_array):
                    # If all values are 0, delete the array
                    del my_array

                else:
                    params, covariance = curve_fit(two_gaussians, x, profile[k][j], p0=initial_guess)

                    # Extract the fitted parameters
                    a1_fit, b1_fit, c1_fit, a2_fit, b2_fit, c2_fit = params

                    # Generate the fitted curve
                    y_fit = two_gaussians(x, a1_fit, b1_fit, c1_fit, a2_fit, b2_fit, c2_fit)

                    diameter.append(-params[1]+params[4])
                    NPC_ID.append(k)
            except: 
                pass
    return(diameter,NPC_ID)

diam_ctrl = two_gauss_fit((1e4, -50, 20, 5e4, 50, 20),profile_control)
dfdiam = pd.DataFrame()
dfdiam['NPC diameter'] = diam_ctrl[0]
dfdiam['NPC id'] = diam_ctrl[1]
dfdiam['Condition'] = 'Control'
#print(dfdiam)

diam_hyper = two_gauss_fit((1e4, -50, 20, 5e4, 50, 20),profile_hyper)
dfdiam_hyper = pd.DataFrame()
dfdiam_hyper['NPC diameter'] = diam_hyper[0]
dfdiam_hyper['NPC id'] = diam_hyper[1]
dfdiam_hyper['Condition'] = 'Hypertonic'
#print(dfdiam_hyper)

df_final = pd.concat([dfdiam,dfdiam_hyper],ignore_index=True)
print(df_final)

In [ ]:
import seaborn as sns
sns.boxplot(x="NPC id", y="NPC diameter",
            data=df_final[df_final["Condition"]=="Control"])

In [ ]:
sns.boxplot(x="NPC id", y="NPC diameter",
            data=df_final[df_final["Condition"]=="Hypertonic"])

In [ ]:
sns.boxplot(x="Condition", y="NPC diameter", hue = "Condition",
            data=df_final)